# Understanding TensorRT - Convert Graph

## Convert the Model
1. checkpoint -> Frozen TensorFlow graph  
2. Frozen TensorFlow graph -> TFLite  
3a. TFLite -> EdgeTPU (Google Coral)  
3b. TFLite -> TensorRT (NVIDIA GPU)  

See TrainModel_Step3_TrainingJob.ipynb  
In this notebook, you'll see a simple script /tasks/covert_checkpoint_to_edgetpu_flight.sh  
that does all of this for you.

This notebook will look at 
- Individual Steps : this is here to help you debug and understand  
- Simple - just use the script : this is here if you know what your doing and everything works fine  


In [ ]:
import os
import sys

In [ ]:
import tensorflow as tf

print (tf.__version__)

## Individual Steps

In [ ]:
# GLOBALS so work with the bash scripts
PROJECT_DIR = os.getcwd()
TASKS_DIR = os.path.join(PROJECT_DIR, 'tasks')
CODE_DIR = os.path.join(PROJECT_DIR, 'code')
TFLITE_DIR = os.path.join(PROJECT_DIR, 'tflite_model')
TENSORFLOW_DIR = os.path.join(PROJECT_DIR, 'tensorflow_model')
MODEL_RESEARCH_DIR = os.path.join(PROJECT_DIR, 'code/models/research')
SLIM_DIR = os.path.join(MODEL_RESEARCH_DIR, 'slim')
TRAINED_DIR = os.path.join(PROJECT_DIR, 'trained_model')


In [ ]:
# add to the Python path
# without this PATH append
# it won't find nets - in the /slim directory
sys.path.append(MODEL_RESEARCH_DIR)
sys.path.append(SLIM_DIR)

In [ ]:
! echo {CODE_DIR}
! ls {MODEL_RESEARCH_DIR}/object_detection/export_inference_graph.py

In [ ]:
ls {MODEL_RESEARCH_DIR}

In [ ]:
! python ${MODEL_RESEARCH_DIR}object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path={CODE_DIR}/{pipeline_config}  --trained_checkpoint_prefix={TRAINED_DIR}/model.ckpt-{ckpt_number} --output_directory={TENSORFLOW_DIR}

## Simple - just use the script

In [ ]:
#### Convert (Trained) Model Checkpoint to a tflite Model


ALSO NOTE:  the script is named convert_checkpoint_to_edge_tflite.sh  
Well... the name is no longer totally accurate
- I took this from the original Coral TPU tutorial
- Another step is required for compiling for the EdgeTPU (not really relevant here since we are confined to AWS where there is no TPU -- so we skip that stuff)
- And, I added a step that converts the checkpoint to a TENSORFLOW frozen graph 
  - note that this generates a frozen_inference_graph.pb
  - it ALSO generates a saved model graph.pb  
  THESE (frozen graph & saved model) are NOT the SAME!!  
  
https://stackoverflow.com/questions/46547319/error-when-parsing-graph-def-from-string

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/exporting_models.md

In [ ]:
! ls code/model

In [ ]:
# WAKE UP - make sure the checkpoint num == hyperparameters/num_train_steps
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir("tasks")  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


In [ ]:
# https://www.tensorflow.org/lite/convert/python_api#using_the_interpreter_from_a_model_file_

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model('code/model')
tflite_model = converter.convert()